In [6]:
!pip install astral

In [32]:
import csv
import requests
from datetime import datetime

def fetch_prayer_times(year, latitude, longitude, method, csv_file_name):
    # API Endpoint
    base_url = "https://api.aladhan.com/v1/calendar"

    # Open CSV file for writing
    with open(csv_file_name, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Date', 'Fajr', 'Sunrise', 'Dhuhr', 'Asr', 'Maghrib', 'Isha'])  # Header row

        # Loop through all months
        for month in range(1, 13):
            # API Request for the month
            params = {
                "latitude": latitude,
                "longitude": longitude,
                "method": method,
                "month": month,
                "year": year
            }
            response = requests.get(base_url, params=params)
            data = response.json()

            # Check if the response is valid
            if data.get("code") == 200:
                for day in data["data"]:
                    # Format date as DD-MM-YYYY
                    date = datetime.strptime(day["date"]["gregorian"]["date"], '%d-%m-%Y').strftime('%d-%m-%Y')
                    timings = day["timings"]

                    # Extract prayer times, add seconds ":00"
                    fajr = timings["Fajr"].split(" ")[0]
                    sunrise = timings["Sunrise"].split(" ")[0]
                    dhuhr = timings["Dhuhr"].split(" ")[0]
                    asr = timings["Asr"].split(" ")[0]
                    maghrib = timings["Maghrib"].split(" ")[0]
                    isha = timings["Isha"].split(" ")[0]

                    # Write row to CSV
                    writer.writerow([date, fajr, sunrise, dhuhr, asr, maghrib, isha])
            else:
                print(f"Error fetching data for {month}/{year}: {data.get('data')}")

    print(f"Prayer times for {year} saved to {csv_file_name}!")

# Example Usage
fetch_prayer_times(
    year=2025,
    latitude=6.9271,  # Colombo, Sri Lanka
    longitude=79.8612,
    method=3,  # Calculation Method (2 = Muslim World League)
    csv_file_name='prayer_times_2025.csv'
)

Prayer times for 2025 saved to prayer_times_2025.csv!


In [35]:
import csv
from datetime import datetime, timedelta

def csv_to_ics(csv_file_name, ics_file_name):
    # Read the CSV file
    with open(csv_file_name, mode='r') as csv_file:
        reader = csv.reader(csv_file)
        next(reader)  # Skip header row

        # Create and open the .ics file for writing
        with open(ics_file_name, mode='w') as ics_file:
            # Write the ICS header
            ics_file.write("BEGIN:VCALENDAR\n")
            ics_file.write("VERSION:2.0\n")
            ics_file.write("CALSCALE:GREGORIAN\n")

            # Process each row in the CSV
            for row in reader:
                date = row[0]  # Date in DD-MM-YYYY format
                fajr = row[1]
                sunrise = row[2]
                dhuhr = row[3]
                asr = row[4]
                maghrib = row[5]
                isha = row[6]

                # Convert the date to YYYYMMDD format
                try:
                    event_date = datetime.strptime(date, '%d-%m-%Y')
                except ValueError:
                    print(f"Skipping invalid date format: {date}")
                    continue

                # Prayer times in order
                prayer_times = [fajr, sunrise, dhuhr, asr, maghrib, isha]
                prayer_names = ['Fajr', 'Sunrise', 'Dhuhr', 'Asr', 'Maghrib', 'Isha']

                # Loop through each prayer time
                for i, (prayer, time) in enumerate(zip(prayer_names, prayer_times)):
                    if not time:  # Skip empty prayer times
                        print(f"Skipping missing time for {prayer} on {date}")
                        continue

                    # Format the time as HH:MM (without modifying it)
                    try:
                        prayer_time = datetime.strptime(time, '%H:%M')
                    except ValueError:
                        print(f"Skipping invalid time format: {time} for {prayer} on {date}")
                        continue

                    # Calculate the Imam Jama'ah time by adding 10 minutes
                    imam_jamaah_time = prayer_time + timedelta(minutes=10)

                    # Format the original prayer time as HHMMSS (with seconds)
                    event_time = prayer_time.strftime('%H%M') + '00'

                    # Format the start time as YYYYMMDDTHHMMSS
                    start_time = event_date.strftime('%Y%m%d') + "T" + event_time

                    # Craft the event description with the Imam Jama'ah time (10 minutes later)
                    description = f"Imam Jama'ah for {prayer} will be in 10 minutes at {imam_jamaah_time.strftime('%I:%M %p')}"

                    # Write the event details to the ICS file
                    ics_file.write(f"BEGIN:VEVENT\n")
                    ics_file.write(f"SUMMARY:{prayer} Prayer\n")
                    ics_file.write(f"DTSTART:{start_time}\n")
                    ics_file.write(f"DESCRIPTION:{description}\n")

                    # Set reminder (trigger at the same time as prayer)
                    ics_file.write(f"BEGIN:VALARM\n")
                    ics_file.write(f"ACTION:DISPLAY\n")
                    ics_file.write(f"DESCRIPTION:Reminder for {prayer} Prayer\n")
                    ics_file.write(f"TRIGGER:-PT0M\n")  # Reminder 0 minutes before (exact time)
                    ics_file.write(f"END:VALARM\n")

                    ics_file.write(f"END:VEVENT\n")

            # Write the ICS footer
            ics_file.write("END:VCALENDAR\n")

    print(f"Converted CSV to ICS file: {ics_file_name}")

# Example Usage
csv_to_ics('/content/prayer_times_2025.csv', '/content/prayer_times_2025.ics')

Converted CSV to ICS file: /content/prayer_times_2025.ics
